In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import glob
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

%matplotlib inline

spotify = pd.read_csv(
    "data/spotify_data.csv",
    names=["end_time", "artist_name", "track_name", "time_played"],
    header=0,
)

spotify["end_time"] = pd.to_datetime(spotify["end_time"], utc=True)
spotify["time_played"] = pd.to_timedelta(spotify["time_played"], unit="ms")

spotify["start_time"] = spotify["end_time"] - spotify["time_played"]

print(f"spotify.shape: {spotify.shape}")
print(f"spotify.columns: {list(spotify.columns)}")

face = pd.read_csv("data/emotion_data.csv")
face["timestamp"] = pd.to_datetime(face["timestamp"], utc=True)

print(f"\nface.shape: {face.shape}")
print(f"face.columns: {list(face.columns)}")

spotify.shape: (28494, 5)
spotify.columns: ['end_time', 'artist_name', 'track_name', 'time_played', 'start_time']

face.shape: (42165, 7)
face.columns: ['timestamp', 'angry', 'scared', 'happy', 'sad', 'surprised', 'neutral']


<IPython.core.display.Javascript object>

In [3]:
# INNER JOIN dfs BETWEEN datetimes
# pandas doesnt offer feature afaik and dont want to learn a new pkg
# its... pretty slow....

joined_rows = []
for i, row in tqdm(face.iterrows(), total=face.shape[0]):
    # fmt: off
    time_filter = ((spotify["start_time"] < row["timestamp"]) & 
                   (spotify["end_time"] > row["timestamp"]))
    # fmt: on

    to_join = spotify[time_filter]

    if to_join.shape[0] < 1:
        continue
    elif to_join.shape[0] > 1:
        # cop out and going to take first if multiple
        to_join = to_join.iloc[[0], :]

    row_df = pd.DataFrame(row).T

    row_df.index = [i]
    to_join.index = [i]

    joined_row = pd.concat((row_df, to_join), axis=1)
    joined_rows.append(joined_row)


joined_df = pd.concat(joined_rows)
joined_df = joined_df.reset_index(drop=True)
joined_df = joined_df.dropna()

joined_df.shape

(14857, 12)

<IPython.core.display.Javascript object>

In [4]:
joined_df.to_csv("data/joined_data.csv", index=False)

<IPython.core.display.Javascript object>